# 如何从文件运行一个具有语义技能（函数）

语义技能是语义函数的集合，其中每个函数都是用自然语言定义的，可以通过文本文件提供。

请参阅我们的术语表，了解这些术语的深入指南。

存储库在samples文件夹下包含一些示例。

例如，以下是FunSkill技能的Joke函数部分:

WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.

+++++
{{$input}}
+++++

请注意特殊的{{$input}}标记，这是一个在调用函数时自动传递的变量，通常称为“函数参数”。

稍后我们将探讨函数如何接受多个变量，以及如何调用其他函数。

在同一文件夹中，您会注意到第二个配置。json文件。该文件是可选的，用于为大型语言模型(如Temperature, TopP, Stop Sequences等)设置一些参数。

{
  "schema": 1,
  "type": "completion",
  "description": "Generate a funny joke",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  }
}

给定这些文件定义的语义函数，下面是加载和使用基于文件的语义函数的方法。

像往常一样加载和配置内核，同时加载在安装笔记本中定义的[AI设置](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.14.547.1-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;

var kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService(model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.14.547.1-preview

导入技能和所有的函数：

In [2]:
// note: using skills from the repo
var skillsDirectory = System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "skills");

var funSkillFunctions = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");